<a href="https://colab.research.google.com/github/GalindoD/Backtester_Stocks/blob/main/Backtester_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Requirements

In [1]:
pip install --upgrade yfinance

In [2]:
pip install backtrader yfinance matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 4.8 MB/s eta 0:00:00


In [3]:

import pandas as pd
import numpy as np
import plotly.express as px
import backtrader as bt
import yfinance as yf
import matplotlib.pyplot as plt
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3995    0  3995    0     0  11073      0 --:--:-- --:--:-- --:--:-- 11097
100  517k  100  517k    0     0   494k      0  0:00:01  0:00:01 --:--:-- 1019k
--2025-06-03 17:41:22--  https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
Resolving anaconda.org (anaconda.org)... 104.19.145.37, 104.19.144.37, 2606:4700::6813:9125, ...
Connecting to anaconda.org (anaconda.org)|104.19.145.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5f738b4b78d4370a69f82984/673b62920c609ff6945bb24d?response-content-disposition=attachment%3B%20filename%3D%22ta-lib-0.5.1-py311h9ecbd09_0.conda%22%3B%20filename%2A%3DUTF-8%27%27ta-lib-0.5.1-py311h9ecbd09_0.conda&response-content-type=application%2Foctet-stream&X-Amz-Algorithm=

#Functions for Data gathering and Computing Buy Signals

In [12]:
def gatherdata(ticker, periodicity):
  data = yf.download(
      tickers = ticker,
      #start="2020-01-06",
      #end="2025-05-26",
      period="max",
      interval=periodicity,
      ignore_tz=True,
      auto_adjust=True)
  return data

def createdf(data, fastma, slowma, fastmacd, slowmacd, signal):
  data_df = data.copy()
  data_df.columns = ["close", "high", "low", "open", "volume"]
  data_df.drop(columns=["volume", "high", "low", "open"], inplace=True)

  data_df["R"] = data_df.close.pct_change().fillna(0)
  data_df["2R"] = data_df.R * 2
  data_df["3R"] = data_df.R * 3

  #Calculating values of Indicators
  data_df["slow_ma"] = talib.SMA(data_df.close, slowma)
  data_df["fast_ma"] = talib.SMA(data_df.close, fastma)
  data_df["macd"], data_df["signal"], data_df["histogram"] = talib.MACD(data_df.close, fastperiod=fastmacd, slowperiod=slowmacd, signalperiod=signal)

  #Delete First NA rows, resultring form lack of previous data
  data_df.dropna(inplace=True)
  return data_df

In [13]:

def signalcompute(data_df):
  data_df = data_df.assign(
      macdbuy = lambda x: np.where( x.macd > x.signal ,1,0)
      )
  data_df = data_df.assign(
      fastmabuy = lambda x: np.where( x.close > x.fast_ma ,1,0)
      )
  data_df = data_df.assign(
      slowmabuy = lambda x: np.where( x.close > x.slow_ma ,1,0)
      )
  return data_df

In [61]:
def compute_strategy(data_df, strategy, liquid_return):
  data_df["buy"] = 0
  match strategy:
    case 1:
      #Strategy 1: MACD and MA, buy both signals 1 and sell both signals 0
      for ei in data_df.index:
        if ( data_df.loc[ei, "macdbuy"] == 1 and data_df.loc[ei, "fastmabuy"]  == 1):
          data_df.loc[ei, "buy"] = 1
        elif ((data_df.loc[ei, "macdbuy"] == 1 or data_df.loc[ei, "fastmabuy"] == 1) and data_df.loc[eip, "buy"] == 1):
          #previous_index = df.index.shift(1, freq='D')
          data_df.loc[ei, "buy"] = 1
        else:
          data_df.loc[ei, "buy"] = 0
        eip = ei
    case 2:
      #Strategy 2: MACD and fastMA, buy both signals 1 and sell when either signal is 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1):
            data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 3:
      #Strategy 3: Only MACD
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 ):
            data_df.loc[i, "buy"] = 1
        else:
            data_df.loc[i, "buy"] = 0
    case 4:
      #Strategy 4: MACD and SlowMA and FastMA
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1 and data_df.loc[i, "slowmabuy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 5:
      #Strategy 5: SlowMA Only
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "slowmabuy"] == 1 ):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0

  #Shift
  data_df["buyshift"] = data_df["buy"].shift(1, fill_value=0)

  #Buying according to strategy
  data_df["R_strategy"] = data_df.R * data_df.buyshift
  data_df['R_strX2'] = data_df.R_strategy * 2
  data_df['R_strX3'] = data_df.R_strategy * 3

  #Assuming 5% return on liquid
  #data_df['R_strategy'] = data_df['R_strategy'].replace([0], liquid_return)

  #Add Transaction cost
  data_df['R_strategy'] = [data_df.loc[ei, 'R_strategy'] - trsncost if data_df.loc[ei, 'buy'] != data_df.loc[ei, 'buyshift'] else data_df.loc[ei, 'R_strategy'] for ei in data_df.index]
  data_df['R_strX2'] = [data_df.loc[ei, 'R_strX2'] - trsncost if data_df.loc[ei, 'buy'] != data_df.loc[ei, 'buyshift'] else data_df.loc[ei, 'R_strX2'] for ei in data_df.index]
  data_df['R_strX3'] = [data_df.loc[ei, 'R_strX3'] - trsncost if data_df.loc[ei, 'buy'] != data_df.loc[ei, 'buyshift'] else data_df.loc[ei, 'R_strX3'] for ei in data_df.index]

  #Add leveraged columns (TQQ & QLD)


  return data_df


#Main Cell for declaring variables and strategies

In [105]:
#Main Cell


#Variables
ticker = "QQQ"

trsncost = 0.05

#periodicity = "1wk"
periodicity = "1d"

if periodicity == "1wk":
  liquid_return = 0.000958
elif periodicity == "1d":
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0.
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0.
#Strategy 3: Only MACD.
#Strategy 4: MACD and 40SlowMA and FastMA.
strategy = 5

slowma = 50
fastma = 40
fastmacd = 12
slowmacd = 26
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, liquid_return)




[*********************100%***********************]  1 of 1 completed


In [106]:
data_df.tail(500)

,close,R,2R,3R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,buyshift,R_strategy,R_strX2,R_strX3
Date,,,,,,,,,,,,,,,,,
2023-06-06,350.343201,-0.000169,-0.000338,-0.000507,321.404830,323.849257,8.943249,7.465824,1.477425,1,1,1,1,1,-0.000169,-0.000338,-0.000507
2023-06-07,344.399475,-0.016965,-0.033931,-0.050896,322.195876,324.663555,8.489023,7.670463,0.818560,1,1,1,1,1,-0.016965,-0.033931,-0.050896
2023-06-08,348.674561,0.012413,0.024826,0.037239,323.104809,325.653595,8.377440,7.811859,0.565581,1,1,1,1,1,0.012413,0.024826,0.037239
2023-06-09,350.007507,0.003823,0.007646,0.011469,323.929820,326.525652,8.300879,7.909663,0.391217,1,1,1,1,1,0.003823,0.007646,0.011469
2023-06-12,355.921600,0.016897,0.033794,0.050691,324.814663,327.560371,8.618079,8.051346,0.566733,1,1,1,1,1,0.016897,0.033794,0.050691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-28,518.909973,-0.004432,-0.008864,-0.013296,478.547354,478.167749,12.451854,12.239098,0.212756,1,1,1,1,1,-0.004432,-0.008864,-0.013296
2025-05-29,519.929993,0.001966,0.003931,0.005897,479.469285,479.348499,12.175851,12.226448,-0.050598,0,1,1,1,1,0.001966,0.003931,0.005897
2025-05-30,519.109985,-0.001577,-0.003154,-0.004731,480.248006,480.422498,11.755439,12.132246,-0.376807,0,1,1,1,1,-0.001577,-0.003154,-0.004731


#Plot last N values

In [107]:
last_200 = data_df.tail(100)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column, shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    #subplot_titles=("Price and Moving Averages", "MACD")
                    )

# Row 1: Price and Moving Averages
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['close'],
                         mode='lines', name='Close'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['slow_ma'],
                         mode='lines', name='SlowMA'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['fast_ma'],
                         mode='lines', name='FastMA'),
              row=1, col=1)

# Row 2: MACD and Signal Line
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['macd'],
                         mode='lines', name='MACD'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['signal'],
                         mode='lines', name='Signal'),
              row=2, col=1)

# Horizontal line at y=0 on MACD plot
fig.add_shape(type="line",
              x0=last_200.index.min(), x1=last_200.index.max(),
              y0=0, y1=0,
              line=dict(color="gray", width=2),
              row=2, col=1)

# Layout and labels
fig.update_layout(height=600, width=900, showlegend=True,
                  title_text=f"Price, MA, MACD: {ticker}.")

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="MACD", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()

#Plot returns

In [108]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strX3"]]).prod()-1
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

Return of Holding:  1147.0043549268685
Return of Strategy:  -0.9999980218693358
The return of this strategy is  -0.0009 x of holding. Ticker:  QQQ


In [109]:
px.line(100 * (1 + data_df[["R","R_strX3"]]).cumprod(), title=f"Total Return: {ticker} {periodicity} -> FMCD: {fastmacd} SMCD: {slowmacd} SIG: {signal}.")

In [110]:
data_df['difference'] = data_df['buy'] != data_df['buyshift']

In [111]:
print(data_df['difference'].sum())

415


#Loop cell for finding best parameters

In [ ]:
#Loop Cell la buena


#Variables
ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000484
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9
fastmacd = 8
slowmacd = 17
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

row_name = "slowmacd_" + str(slowmacd)
column_name = "fastmacd_" + str(fastmacd)

returns_df = pd.DataFrame({column_name: [0]}, index=[row_name])

for slowmacd in range(17, 29):
  row_name = "slowmacd_" + str(slowmacd)
  #returns_df.loc[row_name, column_name] = pd.DataFrame({'Values': [row_name]})

  for fastmacd in range(7, 15):
    #Create dataframe
    data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

    #Compute the signal
    data_df = signalcompute(data_df)

    #Choose and compute strategy
    data_df = compute_strategy(data_df, strategy, days, liquid_return)

    #Total Return with transaction costs
    retvals = (100 * (1+data_df[["R_strategy"]]).prod()-1).item()
    column_name = "fastmacd_" + str(fastmacd)
    returns_df.loc[row_name, column_name] =  retvals

[*********************100%***********************]  1 of 1 completed


End
End
End
End


<ipython-input-11-c9a84258d6c4>:59: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37928.513764057316' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



End
End
End


KeyboardInterrupt: 

In [ ]:
returns_df

In [ ]:
fig = px.imshow(returns_df, text_auto=True, aspect="auto")
fig.show()

#Another cell for testing and computing Misc

In [ ]:
#Main Cell


#Variables
#ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"
ticker = "SOL-USD"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000684
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, days, liquid_return)




[*********************100%***********************]  1 of 1 completed

End


In [ ]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1
print("With transaction cost:")
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

With transaction cost:
Return of Holding:  6612.76163090801
Return of Strategy:  21673.264728914146
The return of this strategy is  3.2775 x of holding. Ticker:  SOL-USD


In [ ]:
px.line(100 * (1 + data_df[["R","R_strategy"]]).cumprod(), title=f"Total Return: {ticker} {periodicity} -> FMCD: {fastmacd} SMCD: {slowmacd} SIG: {signal}.")

In [ ]:
data_df.tail(20)

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy
Date,,,,,,,,,,,,
2025-01-24,253.361221,0.011073,177.602727,211.329198,14.352357,11.999282,2.353075,1,1,1,0,0.000684
2025-01-29,227.944931,-0.100316,179.753226,212.741229,13.876844,12.374794,1.502049,1,1,1,1,-0.102316
2025-02-03,216.149582,-0.051746,181.180155,211.990421,11.793956,12.258627,-0.464671,0,1,1,1,-0.051746
2025-02-08,199.599899,-0.076566,181.851213,212.529593,8.051108,11.417123,-3.366015,0,0,1,0,-0.001316
2025-02-13,194.460953,-0.025746,182.090585,212.227780,4.624107,10.058520,-5.434413,0,0,1,0,0.000684
2025-02-18,169.084290,-0.130497,182.501736,210.033405,-0.979144,7.850987,-8.830131,0,0,0,0,0.000684
2025-02-23,168.038116,-0.006187,183.086887,205.174924,-5.007526,5.279284,-10.286810,0,0,0,0,0.000684
2025-02-28,148.030014,-0.119069,183.126281,201.475397,-10.161550,2.191117,-12.352668,0,0,0,0,0.000684
2025-03-05,146.267303,-0.011908,183.238001,197.352267,-13.758997,-0.998905,-12.760092,0,0,0,0,0.000684


In [ ]:
last_200 = data_df.tail(200)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column, shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    #subplot_titles=("Price and Moving Averages", "MACD")
                    )

# Row 1: Price and Moving Averages
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['close'],
                         mode='lines', name='Close'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['slow_ma'],
                         mode='lines', name='SlowMA'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['fast_ma'],
                         mode='lines', name='FastMA'),
              row=1, col=1)

# Row 2: MACD and Signal Line
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['macd'],
                         mode='lines', name='MACD'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['signal'],
                         mode='lines', name='Signal'),
              row=2, col=1)

# Horizontal line at y=0 on MACD plot
fig.add_shape(type="line",
              x0=last_200.index.min(), x1=last_200.index.max(),
              y0=0, y1=0,
              line=dict(color="gray", width=2),
              row=2, col=1)

# Layout and labels
fig.update_layout(height=600, width=900, showlegend=True,
                  title_text=f"Price, MA, MACD: {ticker}.")

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="MACD", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()

#Test code

In [ ]:
#Main Cell


#Variables
#ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"
ticker = "SOL-USD"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000684
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, days, liquid_return)

[*********************100%***********************]  1 of 1 completed

End


In [ ]:
SOLdf

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy
Date,,,,,,,,,,,,
2021-05-08,45.561531,-0.008102,11.838759,31.514047,10.712764,7.695981,3.016782,1,1,1,0,0.000684
2021-05-13,40.997852,-0.100165,12.819019,34.247702,10.204955,8.197776,2.007179,1,1,1,1,-0.102165
2021-05-18,55.910961,0.363753,14.180760,37.953317,11.445426,8.847306,2.598120,1,1,1,1,0.363753
2021-05-23,24.688747,-0.558427,14.763172,38.166964,8.243364,8.726518,-0.483154,0,0,1,1,-0.558427
2021-05-28,29.039276,0.176215,15.438438,38.367493,6.350466,8.251307,-1.900841,0,0,1,0,-0.001316
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-02,117.757629,-0.090135,182.982515,139.955148,-23.406126,-17.083395,-6.322731,0,0,0,0,0.000684
2025-04-07,106.902779,-0.092180,182.084177,131.796786,-24.706209,-18.607958,-6.098251,0,0,0,0,0.000684
2025-04-12,132.260559,0.237204,181.568387,128.133678,-21.967956,-19.279957,-2.687999,0,1,0,0,0.000684


In [ ]:
BTCdf

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy
Date,,,,,,,,,,,,
2021-05-08,58803.777344,0.028033,38383.356006,56809.437500,4844.755572,6523.879072,-1679.123500,0,1,1,0,0.000684
2021-05-13,49716.191406,-0.154541,39300.481445,56503.630078,3535.296883,5926.162634,-2390.865751,0,0,1,0,0.000684
2021-05-18,42909.402344,-0.136913,40034.553442,55019.550391,1708.172763,5082.564660,-3374.391897,0,0,1,0,0.000684
2021-05-23,34770.582031,-0.189675,40550.475317,52736.219531,-603.518221,3945.348084,-4548.866305,0,0,0,0,0.000684
2021-05-28,35697.605469,0.026661,41059.607568,50473.584766,-2094.987529,2737.280961,-4832.268491,0,0,0,0,0.000684
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-02,82485.710938,-0.022138,86407.344043,86554.630469,-2805.039310,-819.081185,-1985.958124,0,0,0,0,0.000684
2025-04-07,79235.335938,-0.039405,86814.716016,84860.660937,-3312.480657,-1317.761080,-1994.719577,0,0,0,0,0.000684
2025-04-12,85287.109375,0.076377,87339.344531,83776.817187,-2854.417721,-1625.092408,-1229.325313,0,1,0,0,0.000684


In [ ]:
BTCdf.rename(columns={"buy": "BTCbuy"}, inplace=True)

In [ ]:
BTCdf

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,BTCbuy,R_strategy
Date,,,,,,,,,,,,
2021-05-08,58803.777344,0.028033,38383.356006,56809.437500,4844.755572,6523.879072,-1679.123500,0,1,1,0,0.000684
2021-05-13,49716.191406,-0.154541,39300.481445,56503.630078,3535.296883,5926.162634,-2390.865751,0,0,1,0,0.000684
2021-05-18,42909.402344,-0.136913,40034.553442,55019.550391,1708.172763,5082.564660,-3374.391897,0,0,1,0,0.000684
2021-05-23,34770.582031,-0.189675,40550.475317,52736.219531,-603.518221,3945.348084,-4548.866305,0,0,0,0,0.000684
2021-05-28,35697.605469,0.026661,41059.607568,50473.584766,-2094.987529,2737.280961,-4832.268491,0,0,0,0,0.000684
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-02,82485.710938,-0.022138,86407.344043,86554.630469,-2805.039310,-819.081185,-1985.958124,0,0,0,0,0.000684
2025-04-07,79235.335938,-0.039405,86814.716016,84860.660937,-3312.480657,-1317.761080,-1994.719577,0,0,0,0,0.000684
2025-04-12,85287.109375,0.076377,87339.344531,83776.817187,-2854.417721,-1625.092408,-1229.325313,0,1,0,0,0.000684


In [ ]:
concat_df = pd.concat([SOLdf, BTCdf["BTCbuy"]], axis=1)
print(concat_df)

                 close         R     slow_ma     fast_ma       macd  \
Date                                                                  
2021-05-08   45.561531 -0.008102   11.838759   31.514047  10.712764   
2021-05-13   40.997852 -0.100165   12.819019   34.247702  10.204955   
2021-05-18   55.910961  0.363753   14.180760   37.953317  11.445426   
2021-05-23   24.688747 -0.558427   14.763172   38.166964   8.243364   
2021-05-28   29.039276  0.176215   15.438438   38.367493   6.350466   
...                ...       ...         ...         ...        ...   
2025-04-02  117.757629 -0.090135  182.982515  139.955148 -23.406126   
2025-04-07  106.902779 -0.092180  182.084177  131.796786 -24.706209   
2025-04-12  132.260559  0.237204  181.568387  128.133678 -21.967956   
2025-04-17  134.698746  0.018435  180.970127  128.120420 -19.265001   
2025-04-22  148.739410  0.104238  181.109305  128.792252 -15.228741   

               signal  histogram  macdbuy  fastmabuy  slowmabuy  buy  \
Date

In [ ]:
concat_df

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy,BTCbuy
Date,,,,,,,,,,,,,
2021-05-08,45.561531,-0.008102,11.838759,31.514047,10.712764,7.695981,3.016782,1,1,1,0,0.000684,0
2021-05-13,40.997852,-0.100165,12.819019,34.247702,10.204955,8.197776,2.007179,1,1,1,1,-0.102165,0
2021-05-18,55.910961,0.363753,14.180760,37.953317,11.445426,8.847306,2.598120,1,1,1,1,0.363753,0
2021-05-23,24.688747,-0.558427,14.763172,38.166964,8.243364,8.726518,-0.483154,0,0,1,1,-0.558427,0
2021-05-28,29.039276,0.176215,15.438438,38.367493,6.350466,8.251307,-1.900841,0,0,1,0,-0.001316,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-02,117.757629,-0.090135,182.982515,139.955148,-23.406126,-17.083395,-6.322731,0,0,0,0,0.000684,0
2025-04-07,106.902779,-0.092180,182.084177,131.796786,-24.706209,-18.607958,-6.098251,0,0,0,0,0.000684,0
2025-04-12,132.260559,0.237204,181.568387,128.133678,-21.967956,-19.279957,-2.687999,0,1,0,0,0.000684,0


In [ ]:
#Buying according to strategy
concat_df["R_str2"] = concat_df.R * concat_df.BTCbuy

#Assuming 5% return on liquid
concat_df['R_str2'] = concat_df['R_str2'].replace([0], liquid_return)

In [ ]:
concat_df

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy,BTCbuy,R_str2
Date,,,,,,,,,,,,,,
2021-05-08,45.561531,-0.008102,11.838759,31.514047,10.712764,7.695981,3.016782,1,1,1,0,0.000684,0,0.000684
2021-05-13,40.997852,-0.100165,12.819019,34.247702,10.204955,8.197776,2.007179,1,1,1,1,-0.102165,0,0.000684
2021-05-18,55.910961,0.363753,14.180760,37.953317,11.445426,8.847306,2.598120,1,1,1,1,0.363753,0,0.000684
2021-05-23,24.688747,-0.558427,14.763172,38.166964,8.243364,8.726518,-0.483154,0,0,1,1,-0.558427,0,0.000684
2021-05-28,29.039276,0.176215,15.438438,38.367493,6.350466,8.251307,-1.900841,0,0,1,0,-0.001316,0,0.000684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-02,117.757629,-0.090135,182.982515,139.955148,-23.406126,-17.083395,-6.322731,0,0,0,0,0.000684,0,0.000684
2025-04-07,106.902779,-0.092180,182.084177,131.796786,-24.706209,-18.607958,-6.098251,0,0,0,0,0.000684,0,0.000684
2025-04-12,132.260559,0.237204,181.568387,128.133678,-21.967956,-19.279957,-2.687999,0,1,0,0,0.000684,0,0.000684


In [ ]:
px.line(100 * (1 + concat_df[["R","R_strategy","R_str2"]]).cumprod(), title=f"Total Return: {ticker} {periodicity} -> FMCD: {fastmacd} SMCD: {slowmacd} SIG: {signal}.")

#Another cel for testing another way to iterate dataframes

In [ ]:
def compute_strategy(data_df, strategy, days, liquid_return):
  data_df["buy"] = 0
  match strategy:
    case 1:
      #Strategy 1: MACD and MA, buy both signals 1 and sell both signals 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"]  == 1):
          data_df.loc[i, "buy"] = 1
        elif ((data_df.loc[i, "macdbuy"] == 1 or data_df.loc[i, "fastmabuy"] == 1) and data_df.loc[i - pd.Timedelta(days=days), "buy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 2:
      #Strategy 2: MACD and fastMA, buy both signals 1 and sell when either signal is 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1):
            data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 3:
      #Strategy 3: Only MACD
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 ):
            data_df.loc[i, "buy"] = 1
        else:
            data_df.loc[i, "buy"] = 0
    case 4:
      #Strategy 4: MACD and SlowMA and FastMA
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1 and data_df.loc[i, "slowmabuy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0

  #Shift
  data_df["buyshift"] = data_df["buy"].shift(1, fill_value=0)

  #Buying according to strategy
  data_df["R_strategy"] = data_df.R * data_df.buyshift

  #Assuming 5% return on liquid
  data_df['R_strategy'] = data_df['R_strategy'].replace([0], liquid_return)

  #Adding transaction costs to Strategy
  #for i, row in data_df.iterrows():
  #    try:
  #      if data_df.loc[i, "buy"] != data_df.loc[i + pd.Timedelta(days=days), "buy"]:
  #        data_df.loc[i + pd.Timedelta(days=days), "R_strategy"] = data_df.loc[i + pd.Timedelta(days=days), "R_strategy"] - trsncost
  #     else:
  #        continue
  #    except:
  #      print("End")
  data_df['R_strategy'] = [data_df.loc[ei, 'R_strategy'] - trsncost if data_df.loc[ei, 'buy'] != data_df.loc[ei, 'buyshift'] else data_df.loc[ei, 'R_strategy'] for ei in data_df.index]


  return data_df

In [ ]:
#Main Cell


#Variables
ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"
#ticker = "SOL-USD"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000484
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9 --- OPtimized: Slowmacd = 19
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, days, liquid_return)



[*********************100%***********************]  1 of 1 completed


In [ ]:
data_df

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,buyshift,R_strategy
Date,,,,,,,,,,,,,
2015-03-31,244.223999,-0.017334,313.141575,260.901698,-11.506411,-16.736175,5.229764,1,0,0,0,0,0.000484
2015-04-05,260.597992,0.067045,308.223174,261.229396,-10.140698,-15.417080,5.276381,1,0,0,0,0,0.000484
2015-04-10,236.072006,-0.094114,304.071174,260.808296,-11.956971,-14.725058,2.768087,1,0,0,0,0,0.000484
2015-04-15,223.832993,-0.051844,299.678999,259.318095,-14.458531,-14.671753,0.213222,1,0,0,0,0,0.000484
2015-04-20,224.626007,0.003543,295.917850,255.760497,-15.791398,-14.895682,-0.895717,0,0,0,0,0,0.000484
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-17,84895.750000,-0.004589,87820.855664,83831.689844,-2518.174974,-1803.708921,-714.466053,0,1,0,0,0,0.000484
2025-04-22,93441.890625,0.100666,88605.216016,84569.311719,-1182.317308,-1679.430599,497.113291,1,1,1,1,0,-0.001516
2025-04-27,93754.843750,0.003349,89434.534570,85329.336719,-182.693398,-1380.083158,1197.389761,1,1,1,1,1,0.003349


In [ ]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1
print("With transaction cost:")
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

With transaction cost:
Return of Holding:  40865.57201500536
Return of Strategy:  41377.33610548968
The return of this strategy is  1.0125 x of holding. Ticker:  BTC-USD
